In [2]:
import yfinance as yf
import pandas as pd
from PIL import Image, ImageDraw

# Fetch Apple stock data
apple_stock = yf.Ticker("AAPL")
hist = apple_stock.history(start="2009-01-01", end="2024-10-23")

# Reset index and prepare data
hist.reset_index(inplace=True)
hist['Date'] = hist['Date'].dt.strftime('%Y-%m-%d')
hist['20 Day MA'] = hist['Close'].rolling(window=20).mean()
hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
hist = hist.iloc[20:]  # Dropping rows without 20-day MA

# Function to create and save OHLC image for a given 60-day segment
def create_ohlc_image(AAPL_segment, segment_index):
    # Setup parameters for the image
    bar_width = 3
    spacing = 1
    height_factor = 100  
    volume_height = 20  
    space_between_sections = 10  

    # Calculate image dimensions
    num_bars = len(AAPL_segment)
    image_width = num_bars * (bar_width + spacing)
    max_price = AAPL_segment[['Open', 'High', 'Low', 'Close', '20 Day MA']].max().max()
    min_price = AAPL_segment[['Open', 'High', 'Low', 'Close', '20 Day MA']].min().min()
    max_volume = AAPL_segment['Volume'].max()
    volume_scale = volume_height / max_volume
    image_height = int((max_price - min_price) * height_factor) + volume_height + space_between_sections + 10

    # Create a blank image and get drawing context
    image_black_bg = Image.new("RGB", (image_width, image_height), "black")
    draw_black_bg = ImageDraw.Draw(image_black_bg)

    # Reset index for AAPL_segment to help with plotting
    AAPL_segment = AAPL_segment.reset_index(drop=True)

    # Draw OHLC sticks and volume bars
    for index, row in AAPL_segment.iterrows():
        open_px = int((row['Open'] - min_price) * height_factor)
        high_px = int((row['High'] - min_price) * height_factor)
        low_px = int((row['Low'] - min_price) * height_factor)
        close_px = int((row['Close'] - min_price) * height_factor)

        # Calculate positions
        x_position = index * (bar_width + spacing)
        open_y = image_height - open_px - volume_height - space_between_sections
        high_y = image_height - high_px - volume_height - space_between_sections
        low_y = image_height - low_px - volume_height - space_between_sections
        close_y = image_height - close_px - volume_height - space_between_sections

        # Draw high to low line
        draw_black_bg.line([(x_position + 1, high_y), (x_position + 1, low_y)], fill="white")

        # Draw open tick
        draw_black_bg.line([(x_position, open_y), (x_position + 1, open_y)], fill="white")

        # Draw close tick
        draw_black_bg.line([(x_position + 2, close_y), (x_position + 1, close_y)], fill="white")

        # Draw volume bar
        volume_height_px = int(row['Volume'] * volume_scale)
        volume_top = image_height - volume_height_px
        volume_bottom = image_height
        draw_black_bg.line([(x_position + 1, volume_top), (x_position + 1, volume_bottom)], fill="white")

    # Draw 20-day moving average line
    previous_ma_y = None
    for index, row in AAPL_segment.iterrows():
        if pd.notna(row['20 Day MA']):
            ma_y = image_height - int((row['20 Day MA'] - min_price) * height_factor) - volume_height - space_between_sections
            x_position = index * (bar_width + spacing) + 1
            if previous_ma_y is not None:
                draw_black_bg.line([(x_position - (bar_width + spacing), previous_ma_y), (x_position, ma_y)], fill="white")
            previous_ma_y = ma_y

    # Save the image as PNG
    image_filename = f"AAPL_segment_{segment_index + 1}.png"
    image_black_bg.save(image_filename)
    print(f"Saved {image_filename}")

    # Save data as .dat and .feather files
    dat_filename = f"AAPL_segment_{segment_index + 1}.dat"
    feather_filename = f"AAPL_segment_{segment_index + 1}.feather"
    AAPL_segment.to_csv(dat_filename, sep='\t', index=False)
    AAPL_segment.to_feather(feather_filename)
    print(f"Saved {dat_filename} and {feather_filename}")

# Define the chunk size (20 days)
chunk_size = 20

# Loop over each 60-day chunk and generate an OHLC image
for i in range(0, len(hist), chunk_size):
    AAPL_segment = hist.iloc[i:i + chunk_size]
    
    # Only generate images if there are at least 20 rows in the segment
    if len(AAPL_segment) == chunk_size:
        create_ohlc_image(AAPL_segment, i // chunk_size)


Saved AAPL_segment_1.png
Saved AAPL_segment_1.dat and AAPL_segment_1.feather
Saved AAPL_segment_2.png
Saved AAPL_segment_2.dat and AAPL_segment_2.feather
Saved AAPL_segment_3.png
Saved AAPL_segment_3.dat and AAPL_segment_3.feather
Saved AAPL_segment_4.png
Saved AAPL_segment_4.dat and AAPL_segment_4.feather
Saved AAPL_segment_5.png
Saved AAPL_segment_5.dat and AAPL_segment_5.feather
Saved AAPL_segment_6.png
Saved AAPL_segment_6.dat and AAPL_segment_6.feather
Saved AAPL_segment_7.png
Saved AAPL_segment_7.dat and AAPL_segment_7.feather
Saved AAPL_segment_8.png
Saved AAPL_segment_8.dat and AAPL_segment_8.feather
Saved AAPL_segment_9.png
Saved AAPL_segment_9.dat and AAPL_segment_9.feather
Saved AAPL_segment_10.png
Saved AAPL_segment_10.dat and AAPL_segment_10.feather
Saved AAPL_segment_11.png
Saved AAPL_segment_11.dat and AAPL_segment_11.feather
Saved AAPL_segment_12.png
Saved AAPL_segment_12.dat and AAPL_segment_12.feather
Saved AAPL_segment_13.png
Saved AAPL_segment_13.dat and AAPL_segmen

Saved AAPL_segment_125.png
Saved AAPL_segment_125.dat and AAPL_segment_125.feather
Saved AAPL_segment_126.png
Saved AAPL_segment_126.dat and AAPL_segment_126.feather
Saved AAPL_segment_127.png
Saved AAPL_segment_127.dat and AAPL_segment_127.feather
Saved AAPL_segment_128.png
Saved AAPL_segment_128.dat and AAPL_segment_128.feather
Saved AAPL_segment_129.png
Saved AAPL_segment_129.dat and AAPL_segment_129.feather
Saved AAPL_segment_130.png
Saved AAPL_segment_130.dat and AAPL_segment_130.feather
Saved AAPL_segment_131.png
Saved AAPL_segment_131.dat and AAPL_segment_131.feather
Saved AAPL_segment_132.png
Saved AAPL_segment_132.dat and AAPL_segment_132.feather
Saved AAPL_segment_133.png
Saved AAPL_segment_133.dat and AAPL_segment_133.feather
Saved AAPL_segment_134.png
Saved AAPL_segment_134.dat and AAPL_segment_134.feather
Saved AAPL_segment_135.png
Saved AAPL_segment_135.dat and AAPL_segment_135.feather
Saved AAPL_segment_136.png
Saved AAPL_segment_136.dat and AAPL_segment_136.feather
Save